# Walk-Forward Validation

Perform walk-forward analysis to validate strategy robustness.

**Version**: v1.11.0  
**Architecture**: Modular (lib/validate/, lib/paths/, lib/config/)

## Configuration

Set the strategy name and walk-forward parameters below.

**Note**: This notebook uses the v1.11.0 modular architecture. All imports use canonical paths from `lib/` packages.


In [ ]:
# Configuration
strategy_name = 'spy_sma_cross'  # Change to your strategy
start_date = '2020-01-01'
end_date = None  # None = today
train_period = 252  # Trading days for training (1 year)
test_period = 63    # Trading days for testing (1 quarter)
step_size = 63      # Step size between windows (1 quarter)
objective = 'sharpe'  # Objective metric for optimization


In [ ]:
# Setup
import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Standard library imports
import json
from datetime import datetime

# Third-party imports
import pandas as pd
import numpy as np

# Local imports - lib modules (v1.11.0 modular architecture)
# Validation modules provide walk-forward and Monte Carlo analysis
from lib.paths import get_project_root, get_results_dir
from lib.validate import walk_forward, calculate_walk_forward_efficiency
from lib.config import load_strategy_params
from lib.validation import validate_bundle
from lib.calendars import get_calendar_for_asset_class

In [ ]:
# Pre-validation Checks
print("Pre-walk-forward validation:")
try:
    # Load strategy to get asset class
    params = load_strategy_params(strategy_name)
    asset_class = params.get('strategy', {}).get('asset_class', 'equities')
    calendar = get_calendar_for_asset_class(asset_class)
    
    print(f"  ✓ Strategy: {strategy_name}")
    print(f"  ✓ Asset Class: {asset_class}")
    print(f"  ✓ Calendar: {calendar}")
    
    # Validate date range
    if end_date is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
    
    print(f"  ✓ Date Range: {start_date} to {end_date}")
    print(f"  ✓ Train Period: {train_period} days")
    print(f"  ✓ Test Period: {test_period} days")
    
except Exception as e:
    print(f"  ✗ Validation error: {e}")
    raise

# Run walk-forward analysis
print(f"\nRunning walk-forward analysis for {strategy_name}...")
print(f"Date range: {start_date} to {end_date or 'today'}")
print(f"Train period: {train_period} days")
print(f"Test period: {test_period} days")
print(f"Step size: {step_size} days")

try:
    results = walk_forward(
        strategy_name=strategy_name,
        start_date=start_date,
        end_date=end_date,
        train_period=train_period,
        test_period=test_period,
        objective=objective
    )
    
    print(f"\n✓ Walk-forward analysis complete")
    if 'out_sample_results' in results:
        print(f"  Periods: {len(results['out_sample_results'])}")
    elif 'windows' in results:
        print(f"  Windows: {len(results['windows'])}")
    
except Exception as e:
    print(f"✗ Walk-forward analysis failed: {e}")
    print(f"  Check that strategy exists and date range is valid")
    raise


In [ ]:
# Display walk-forward validation results
if results:
    print("\n" + "=" * 60)
    print("WALK-FORWARD VALIDATION RESULTS")
    print("=" * 60)
    
    # Handle both old and new result formats
    if 'robustness' in results:
        # Old format with robustness dict
        robustness = results['robustness']
        print("\nRobustness Metrics:")
        print(f"  Walk-Forward Efficiency: {robustness.get('efficiency', 0):.3f}")
        print(f"  Consistency: {robustness.get('consistency', 0):.2%}")
        print(f"  Avg IS Sharpe: {robustness.get('avg_is_sharpe', 0):.3f}")
        print(f"  Avg OOS Sharpe: {robustness.get('avg_oos_sharpe', 0):.3f}")
        print(f"  Std OOS Sharpe: {robustness.get('std_oos_sharpe', 0):.3f}")
        print(f"  Number of Periods: {robustness.get('n_periods', 0)}")
    
    # Overall statistics (new format)
    if 'summary' in results:
        summary = results['summary']
        print("\nOverall Summary:")
        print(f"  Total Windows: {summary.get('total_windows', 0)}")
        print(f"  Average Test Return: {summary.get('avg_test_return', 0):.2%}")
        print(f"  Average Test Sharpe: {summary.get('avg_test_sharpe', 0):.3f}")
        print(f"  Consistency: {summary.get('consistency', 0):.2%}")
    
    # Window-by-window results
    if 'windows' in results and len(results['windows']) > 0:
        print("\nWindow-by-Window Results:")
        windows_df = pd.DataFrame(results['windows'])
        
        # Display key metrics
        display_cols = ['train_start', 'test_start', 'test_end', 
                       'test_return', 'test_sharpe', 'test_max_dd']
        available_cols = [col for col in display_cols if col in windows_df.columns]
        
        if available_cols:
            print(windows_df[available_cols].to_string(index=False))
        
        # Calculate efficiency if available
        try:
            efficiency = calculate_walk_forward_efficiency(results)
            print(f"\nWalk-Forward Efficiency: {efficiency:.3f}")
            if efficiency > 0.5:
                print("  ✓ Good efficiency (OOS performance > 50% of IS)")
            elif efficiency > 0.3:
                print("  ⚠ Moderate efficiency (OOS performance 30-50% of IS)")
            else:
                print("  ✗ Low efficiency (OOS performance < 30% of IS)")
        except Exception as e:
            print(f"  ⚠ Could not calculate efficiency: {e}")
    
    # Save results
    results_dir = get_results_dir() / strategy_name / 'latest'
    if results_dir.exists():
        results_file = results_dir / 'walkforward_results.json'
        try:
            # Convert to JSON-serializable format
            results_serializable = {
                'summary': results.get('summary', {}),
                'windows': [
                    {k: (str(v) if isinstance(v, pd.Timestamp) else v) 
                     for k, v in window.items()}
                    for window in results.get('windows', [])
                ]
            }
            
            with open(results_file, 'w') as f:
                json.dump(results_serializable, f, indent=2, default=str)
            
            print(f"\n✓ Results saved to: {results_file}")
        except Exception as e:
            print(f"  ⚠ Could not save results: {e}")
    
    print("=" * 60)
else:
    print("⚠ No results to display")


In [ ]:
# Display IS vs OOS comparison (if available in old format)
if results and 'in_sample_results' in results and 'out_sample_results' in results:
    is_df = results['in_sample_results']
    oos_df = results['out_sample_results']
    
    print("\nIn-Sample vs Out-of-Sample Comparison:")
    comparison = pd.DataFrame({
        'Period': range(1, len(is_df) + 1),
        'IS Sharpe': is_df['sharpe'].values,
        'OOS Sharpe': oos_df['sharpe'].values,
    })
    print(comparison.to_string(index=False))
elif results and 'windows' in results:
    # New format: extract from windows
    windows_df = pd.DataFrame(results['windows'])
    if 'train_sharpe' in windows_df.columns and 'test_sharpe' in windows_df.columns:
        print("\nIn-Sample vs Out-of-Sample Comparison:")
        comparison = pd.DataFrame({
            'Window': range(1, len(windows_df) + 1),
            'IS Sharpe': windows_df['train_sharpe'].values,
            'OOS Sharpe': windows_df['test_sharpe'].values,
        })
        print(comparison.to_string(index=False))


In [ ]:
# Visualize walk-forward results
if results and 'windows' in results and len(results['windows']) > 0:
    try:
        import matplotlib.pyplot as plt
        
        windows_df = pd.DataFrame(results['windows'])
        
        if 'test_return' in windows_df.columns and 'test_sharpe' in windows_df.columns:
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
            
            # Plot test returns
            ax1.bar(range(len(windows_df)), windows_df['test_return'], 
                   color=['green' if x > 0 else 'red' for x in windows_df['test_return']])
            ax1.set_title('Walk-Forward Test Returns by Window', fontsize=14, fontweight='bold')
            ax1.set_xlabel('Window Number')
            ax1.set_ylabel('Test Return')
            ax1.axhline(y=0, color='black', linestyle='--', linewidth=1)
            ax1.grid(True, alpha=0.3)
            
            # Plot test Sharpe ratios
            ax2.bar(range(len(windows_df)), windows_df['test_sharpe'],
                   color=['green' if x > 0 else 'red' for x in windows_df['test_sharpe']])
            ax2.set_title('Walk-Forward Test Sharpe Ratios by Window', fontsize=14, fontweight='bold')
            ax2.set_xlabel('Window Number')
            ax2.set_ylabel('Test Sharpe Ratio')
            ax2.axhline(y=0, color='black', linestyle='--', linewidth=1)
            ax2.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
        else:
            print("⚠ Required columns not found for visualization")
            print(f"  Available columns: {list(windows_df.columns)}")
            
    except ImportError:
        print("⚠ Matplotlib not available for visualization")
    except Exception as e:
        print(f"⚠ Visualization error: {e}")
elif results and 'out_sample_results' in results:
    # Old format: try to visualize from out_sample_results
    try:
        import matplotlib.pyplot as plt
        
        oos_df = results['out_sample_results']
        if 'sharpe' in oos_df.columns:
            fig, ax = plt.subplots(figsize=(12, 6))
            ax.bar(range(len(oos_df)), oos_df['sharpe'],
                   color=['green' if x > 0 else 'red' for x in oos_df['sharpe']])
            ax.set_title('Walk-Forward Test Sharpe Ratios by Period', fontsize=14, fontweight='bold')
            ax.set_xlabel('Period Number')
            ax.set_ylabel('Test Sharpe Ratio')
            ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
            ax.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
    except ImportError:
        print("⚠ Matplotlib not available for visualization")
    except Exception as e:
        print(f"⚠ Visualization error: {e}")
else:
    print("⚠ No visualization data available")